# DataJoint U24 - Workflow Behavior

First, please install both `element-deeplabcut` and `workflow-deeplabcut` locally. We 
recommend launching a new conda environment and using `pip install -e ./<dir>`. For more
information, see our [install instructions](https://github.com/kabilar/datajoint-elements/blob/main/install.md). 

Next, let's change directory to the main workflow directory.

In [2]:
import os; from pathlib import Path
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='workflow-deeplabcut', ("Please move to the "
                                                              + "workflow directory")

Second, download the example data we'll be using from the DeepLabCut repository. We will
use the [example openfield data](https://github.com/DeepLabCut/DeepLabCut/tree/master/examples/openfield-Pranav-2018-10-30) 
from the DeepLabCut github repository. If you have already cloned this repository, you 
may have this data on your machine already. [This link](https://downgit.github.io/#/home?url=https://github.com/DeepLabCut/DeepLabCut/tree/master/examples/openfield-Pranav-2018-10-30) via [DownGit](https://downgit.github.io/) will start the single-directory download 
automatically. After downloading, please add the path to this directory to the `custom`
field of your datajoint config file as `dlc_root_data_dir`. 

In [3]:
import datajoint as dj; dj.config.load('dj_local_conf.json')
from element_interface.utils import find_full_path
data_dir = find_full_path(dj.config['custom']['dlc_root_data_dir'],
                          'openfield-Pranav-2018-10-30')
assert data_dir.exists(), "Please check the that you have the folder openfield-Pranav"

Later, we'll use the first few seconds of this video as a 'separate session' to model
the pose estimation feature of this pipeline. `ffmpeg` is a dependency of DeepLabCut
that can splice the training video for a demonstration purposes. The command below saves
the first 2 seconds of the training video as a copy.

In [3]:
vid_path = str(data_dir).replace(" ", "\ ") + '/videos/m3v1mp4'
cmd = f'ffmpeg -y -ss 2 -i {vid_path}.mp4 -vcodec copy -acodec copy {vid_path}-copy.mp4'
os.system(cmd)

ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.3)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  liba

0

Now, we can activate the `dlc` schema and import some data from files stored in this
directory under `user_data/<file>.csv`. This includes parameters like shuffle and 
training fraction that DeepLabCut uses.

In [4]:
from workflow_deeplabcut.pipeline import lab, subject, session, dlc
from workflow_deeplabcut.ingest import ingest_subjects, ingest_sessions, ingest_dlc_configs
ingest_subjects(); ingest_sessions(); ingest_dlc_configs()

Connecting cbroz@tutorial-db.datajoint.io:3306

---- Inserting 0 entry(s) into subject ----

---- Inserting 0 entry(s) into session ----

---- Inserting 0 entry(s) into session_directory ----

---- Inserting 0 entry(s) into session_note ----

---- Inserting 0 entry(s) into video_recording ----

---- Inserting 0 entry(s) into video_recording__file ----


For model training, we'll work with the following session and parameters.

In [5]:
train_key={'subject': 'subject6', 'session_datetime': '2021-06-02 14:04:22',
           'camera_id': 1, 'recording_start_time': '2021-06-02 14:07:00'}
dlc.VideoRecording & train_key

subject,session_datetime,camera_id,recording_start_time
subject6,2021-06-02 14:04:22,1,2021-06-02 14:07:00


In [6]:
train_key['paramset_idx']=0
dlc.ConfigParamSet & train_key

paramset_idx,shuffle shuffle number to use (usually 1),train_fraction training fraction,"model_prefix DLC model prefix, often empty","filter_type filter type, blank if none (e.g., median, arima)","track_method tracking method, blank if none (e.g,. box, ellipse)",scorer_legacy legacy naming for DLC < v2.1.0,param_set_hash hash identifying this parameterset
0,1,0.95,,,,0,c70007c1-32b1-ae9b-cb83-7d11cbf78f37


Now, we'll insert this combination into the `TrainingTask` table, and ask DeepLabCut to
train the model for us, for one quick iteration.

In [7]:
train_key['training_id']=1
dlc.TrainingTask.insert1(train_key, skip_duplicates=True)
dlc.TrainingTask()

subject,session_datetime,camera_id,recording_start_time,paramset_idx,training_id
subject6,2021-06-02 14:04:22,1,2021-06-02 14:07:00,0,1


In [8]:
dlc.ModelTraining.train_model(training_id=1,maxiters=1)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'clahe': True,
 'claheratio': 0.1,
 'crop_pad': 0,
 'crop_sampling': 'hybrid',
 'crop_size': [400, 400],
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'edge': False,
 'emboss': {'alpha': [0.0, 1.0], 'embossratio': 0.1, 'strength': [0.5, 1.5]},
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'histeq': True,
 'histeqratio': 0.1,
 'init_weights': '/Users/cb/miniconda3/envs/venv-dlc/lib/python3.8/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'l

Selecting single-animal trainer
Batch Size is 1


/Users/cb/miniconda3/envs/venv-dlc/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2022-02-22 17:11:26.399703: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading ImageNet-pretrained resnet_50
Max_iters overwritten as 1
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/Volumes/GoogleDrive/My Drive/Dev/DeepLabCut/examples/JUPYTER/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 'all_joints': [[0], [1], [2], [3]], 'al

2022-02-22 17:11:36.969368: W tensorflow/core/kernels/queue_base.cc:277] _0_fifo_queue: Skipping cancelled enqueue attempt with queue not closed
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/cb/miniconda3/envs/venv-dlc/lib/python3.8/site-packages/tensorflow/python/client/session.py", line 1380, in _do_call
    return fn(*args)
  File "/Users/cb/miniconda3/envs/venv-dlc/lib/python3.8/site-packages/tensorflow/python/client/session.py", line 1363, in _run_fn
    return self._call_tf_sessionrun(options, feed_dict, fetch_list,
  File "/Users/cb/miniconda3/envs/venv-dlc/lib/python3.8/site-packages/tensorflow/python/client/session.py", line 1456, in _call_tf_sessionrun
    return tf_session.TF_SessionRun_wrapper(self._session, options, feed_dict,
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node fifo_queue_enqueue}}]]

During handling of the above exception, another exception occurred:

Traceback (most recent c

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


In [9]:
dlc.ModelTraining()

subject,session_datetime,camera_id,recording_start_time,paramset_idx,training_id,"snapshot_index_exact latest exact snapshot index (i.e., never -1)",config_template stored full config file
subject6,2021-06-02 14:04:22,1,2021-06-02 14:07:00,0,1,1,=BLOB=


Next, we can optionally describe each of these body parts in our `BodyPart` lookup 
table by providing a list of descriptions in the order given by the PoseEstimation 
method. If you skip this step, the same items will be inserted during the following
step.

In [10]:
from element_deeplabcut.readers.dlc_reader import PoseEstimation
print(PoseEstimation(data_dir).body_parts)
description_list = ['Left Ear', 'Right Ear', 'Snout tip', 'Base of the Tail']
dlc.BodyPart.insert_all_from_model(train_key,skip_duplicates=True,
                                   description_list=description_list)

Index(['leftear', 'rightear', 'snout', 'tailbase'], dtype='object', name='bodyparts')


In [11]:
dlc.BodyPart()

body_part,body_part_description
leftear,Left Ear
rightear,Right Ear
snout,Snout tip
tailbase,Base of the Tail


Now, we'll insert the model into the central `Model` table.

In [12]:
dlc.Model.insert_new_model(train_key,config_paramset_idx=0,model_name="FirstModel",
                           model_description="First inserted model", training_id=1)

The `ModelEval` table runs DeepLabCut's evaluation function and stores the result.

In [13]:
dlc.ModelEval.populate()

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/Users/cb/miniconda3/envs/venv-dlc/lib/python3.8/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 4,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 'shuffle': True,


Running  DLC_resnet50_openfieldOct30shuffle1_1008  with # of training iterations: 1008
This net has already been evaluated!


In [14]:
dlc.ModelEval()

model_name user-friendly model name,train_iterations Training iterations,train_error Train error (px),test_error Test error (px),p_cutoff p-cutoff used,train_error_p Train error with p-cutoff,test_error_p Test error with p-cutoff
FirstModel,1008,11.35,6.04,0.4,9.3,5.1


Finally, we can use this model to conduct pose estimation for separate videos. In this
case, we'll use the `m3v1mp4-copy.mp4` clip we generated earlier. First, this line is
inserted into the `PoseEstimationTask` table before conducting the actual estimation 
via the the `PoseEstimation.populate()` method. 

In [7]:
estim_key = ((dlc.VideoRecording & 'session_datetime>"2021-06-03"').fetch1('KEY'))
estim_key.update((dlc.Model & 'model_name="FirstModel"').fetch1('KEY'))
estim_key['task_mode']='trigger'
dlc.PoseEstimationTask.insert1(estim_key)
dlc.PoseEstimationTask()

DuplicateError: ("Duplicate entry 'subject6-2021-06-03 14:04:22-1-2021-06-04 14:07:00-FirstModel' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [16]:
dlc.PoseEstimation.populate()

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/Users/cb/miniconda3/envs/venv-dlc/lib/python3.8/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 4,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 'shuffle': True,


Using snapshot-1008 for model /Volumes/GoogleDrive/My Drive/Dev/DeepLabCut/examples/JUPYTER/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1
Starting to analyze %  /Volumes/GoogleDrive/My Drive/Dev/DeepLabCut/examples/JUPYTER/openfield-Pranav-2018-10-30/videos/m3v1mp4-copy.mp4
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


In [5]:
dlc.PoseEstimation()

subject,session_datetime,camera_id,recording_start_time,model_name user-friendly model name,post_estimation_time time of generation of this set of DLC results
subject6,2021-06-03 14:04:22,1,2021-06-04 14:07:00,FirstModel,2022-01-26 11:22:34


In [8]:
dlc.PoseEstimation.GetTrajectory(estim_key)

scorer     FirstModel                                                      \
bodyparts     leftear                                rightear               
coords              x           y    z likelihood           x           y   
0           78.326073  102.661049  0.0   0.314506   74.472694  109.366257   
1           68.676651  102.067673  0.0   0.294856   74.428070  109.693573   
2           68.009209  101.714394  0.0   0.386914   72.865013   97.332115   
3           67.158943  101.464378  0.0   0.459625   67.720116   95.434067   
4           66.202774  100.679390  0.0   0.457190   67.565826   94.599884   
...               ...         ...  ...        ...         ...         ...   
2325       356.629333  376.825684  0.0   0.092403  346.879303  381.007050   
2326       350.492767  387.181976  0.0   0.118965  410.790131  397.797607   
2327       351.700409  387.601074  0.0   0.150138  411.026489  397.987823   
2328       354.063324  388.168182  0.0   0.137858  427.673187  413.466095   
2329       432.365479  442.043457  0.0   0.118110  427.488220  413.458435   

scorer                                                                         \
bodyparts                       snout                                tailbase   
coords       z likelihood           x           y    z likelihood           x   
0          0.0   0.239496   71.408234  114.323616  0.0   0.315478   78.313431   
1          0.0   0.236722   71.155952  114.597374  0.0   0.297934   78.757637   
2          0.0   0.217421   71.495941   97.109734  0.0   0.316984   78.588943   
3          0.0   0.228822   68.802917   94.268013  0.0   0.329084   78.494675   
4          0.0   0.219198   68.946259   93.628883  0.0   0.309569   78.469231   
...        ...        ...         ...         ...  ...        ...         ...   
2325       0.0   0.096084  343.775360  384.188934  0.0   0.125122  425.655670   
2326       0.0   0.110812  352.447784  385.325653  0.0   0.144934  418.131439   
2327       0.0   0.121694  353.609650  385.443298  0.0   0.193439  420.859375   
2328       0.0   0.121844  354.449799  385.983337  0.0   0.178257  422.556610   
2329       0.0   0.128129  355.735626  386.126068  0.0   0.160270  436.392853   

scorer                                 
bodyparts                              
coords              y    z likelihood  
0          110.994553  0.0   0.273981  
1          110.869270  0.0   0.284565  
2          111.011925  0.0   0.267579  
3          111.527504  0.0   0.254363  
4          112.223282  0.0   0.229301  
...               ...  ...        ...  
2325       422.168610  0.0   0.212187  
2326       406.169312  0.0   0.195705  
2327       404.963989  0.0   0.217717  
2328       404.818970  0.0   0.243224  
2329       422.403168  0.0   0.222099  

[2330 rows x 16 columns]

# From scratch didactic guide - needs work

This notebook will describe the steps to explore the lab and animal management tables created by the elements.
Prior to using this notebook, please refer to the README for the installation instructions.

Importing the module `workflow_behavior.pipeline` is sufficient to create tables inside the elements. This workflow comes prepackaged with example data and ingestion functions to populate lab, subject, and session tables.

## Workflow architecture

In [ ]:
from element_lab import lab
from element_animal import subject
from element_session import sessions

In [ ]:
lab.Lab()

In [ ]:
dj.Diagram(lab)

In [ ]:
subject.Subject()

In [ ]:
dj.Diagram(subject)

In [ ]:
session.Session()

In [ ]:
dj.Diagram(session)

(Workflow needs continued development to import geotyping tables)

## Explore each table

In [ ]:
# check table definition with describe()
subject.Subject.describe()

## Insert data into Manual and Lookup tables

Tables in this workflow are either manual tables or lookup tables. To insert into these tables, DataJoint provide method `.insert1()` and `insert()`.

In [ ]:
subject.Subject.insert1(
    dict(subject='subject1', sex='M', subject_birth_date='2020-12-30', 
         subject_description='test animal'), skip_duplicates=True)
subject.Subject.insert1(
    ('subject2', 'F', '2020-11-30', 'test animal'), skip_duplicates=True)

`skip_duplicates=True` will prevent an error if you already have data for the primary keys in a given entry.

In [ ]:
subject.Subject()

In [ ]:
# `insert()` takes a list of dicts or tuples
subject.Subject.insert(
    [dict(subject='subject3', sex='F', subject_birth_date='2020-12-30', 
            subject_description='test animal'),
     dict(subject='subject4', sex='M', subject_birth_date='2021-02-12', 
          subject_description='test animal')
    ],
    skip_duplicates=True)
subject.Subject.insert(
    [
        ('subject7', 'U', '2020-08-30', 'test animal'),
        ('subject8', 'F', '2020-09-30', 'test animal')
    ],
    skip_duplicates=True)

In [ ]:
subject.Subject()

For more documentation of insert, please refer to [DataJoint Docs](https://docs.datajoint.io/python/manipulation/1-Insert.html) and [DataJoint playground](https://playground.datajoint.io/)

## Insert into Manual and Lookup tables with Graphical User Interface

DataJoint also provides a Graphical User Interface [DataJoint Labbook](https://github.com/datajoint/datajoint-labbook) to support manual data insertions into DataJoint workflows. ![DataJoint Labbook preview](https://github.com/datajoint/datajoint-labbook/blob/master/docs/sphinx/_static/images/walkthroughDemoOptimized.gif)